In [3]:
import random
import copy
import datetime
from dataclasses import dataclass
from typing import List, Dict, Tuple, Sequence, Callable

In [71]:
@dataclass
class Chromosome:
    genes   : List
    fitness : float


In [72]:
def generate_parent(length: int, geneSet: Sequence, get_fitness: Callable)->Chromosome:
    genes = []
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    fitness = get_fitness(genes)
    return Chromosome(genes, fitness)


In [73]:
def get_fitness(genes: Sequence)->float:
    fitness = 1
    
    for i in range(1, len(genes)):
        if genes[i] > genes[i - 1]:
            fitness += 1
    return fitness

def list_to_string(mlist : List)->str:
    l2 = [f"{x}" for x in mlist]
    return ''.join(l2)

def mutate_genes(genes: List, geneSet: List)->List:
    """Mutate genes target choosing character from geneSet"""

    mutatedGenes         = genes[:]
    index                = random.randrange(0, len(mutatedGenes))
    newGene, alternate   = random.sample(geneSet, 2)
    mutatedGenes[index]  = alternate if newGene == mutatedGenes[index] else newGene
    return mutatedGenes


def mutate_chromosome(geneSet:     Sequence,
                      parent:      Chromosome,
                      get_fitness: Callable)->Chromosome:
    """Generate a mutated chromosome"""
    mutatedGenes = mutate_genes(parent.genes, geneSet)
    fitness      = get_fitness(mutatedGenes)
    return Chromosome(mutatedGenes, fitness)



In [74]:
geneSet = [i for i in range(100)]

def sort_numbers(totalNumbers, imax=1000):
    startTime = datetime.datetime.now()
    random.seed()
    setToSort = generate_parent(totalNumbers, geneSet, get_fitness)
    print(f'Set to Sort = {setToSort}')

    optimalFitness = len(setToSort.genes)
    i = 0
    while setToSort.fitness < optimalFitness and i < imax:
        newGuess = mutate_chromosome(geneSet, setToSort, get_fitness)
        #print(f'new guess : {newGuess}, i = {i}')
        if newGuess.fitness > setToSort.fitness:
            setToSort = Chromosome(newGuess.genes, newGuess.fitness)
            print(f'new guess increased fitness : {setToSort}, i = {i}')
        i+=1
    print(setToSort)
    print(i)


 

In [75]:
sort_numbers(10)

Set to Sort = Chromosome(genes=[24, 33, 4, 97, 14, 25, 54, 71, 77, 13], fitness=7)
new guess increased fitness : Chromosome(genes=[24, 33, 4, 97, 14, 25, 54, 71, 77, 96], fitness=8), i = 7
new guess increased fitness : Chromosome(genes=[24, 33, 50, 97, 14, 25, 54, 71, 77, 96], fitness=9), i = 31
Chromosome(genes=[24, 33, 50, 97, 14, 25, 54, 71, 77, 96], fitness=9)
1000


In [76]:
@dataclass
class Fitness:
    sortCount : int
    gap       : int


In [77]:
@dataclass
class Chromosome:
    genes   : List
    fitness : Fitness


In [78]:
def compare_fitness(f1: Fitness, f2: Fitness)->bool:
    """Returns True if f1 is fitter than f2"""
    if f1.sortCount == f2.sortCount:
        return f1.gap <= f2.gap
    else: 
        return f1.sortCount >= f2.sortCount

In [79]:
def get_fitness(genes : List)->Fitness:
    sortCount = 1
    gap = 0

    for i in range(1, len(genes)):
        if genes[i] > genes[i - 1]:
            sortCount += 1
        else:
            gap += genes[i - 1] - genes[i]
    return Fitness(sortCount, gap)


In [80]:
def sort_numbers(totalNumbers, verbose=True, imax=10000):
    geneSet = [i for i in range(100)]
    startTime = datetime.datetime.now()
    random.seed()
    setToSort = generate_parent(totalNumbers, geneSet, get_fitness)
    print(f'Set to Sort = {setToSort}')

    optimalFitness = Fitness(totalNumbers, 0)
    
    i = 0
    while compare_fitness(setToSort.fitness, optimalFitness) == False and i < imax:
        newGuess = mutate_chromosome(geneSet, setToSort, get_fitness)
        if compare_fitness(newGuess.fitness, setToSort.fitness) == True:
            if verbose:
                print(f'new guess increased fitness  (increased sortCount): {setToSort}, i = {i}')
            setToSort = Chromosome(newGuess.genes, newGuess.fitness)
        
        i+=1
    print(setToSort)
    print(i)


In [81]:
sort_numbers(10, verbose=False)

Set to Sort = Chromosome(genes=[47, 4, 80, 50, 2, 27, 43, 22, 87, 32], fitness=Fitness(sortCount=5, gap=197))
Chromosome(genes=[18, 24, 28, 30, 33, 37, 40, 52, 75, 88], fitness=Fitness(sortCount=10, gap=0))
956


In [82]:
sort_numbers(20, verbose=False)

Set to Sort = Chromosome(genes=[62, 33, 10, 80, 39, 53, 52, 8, 83, 79, 11, 22, 45, 67, 13, 55, 69, 75, 91, 2], fitness=Fitness(sortCount=11, gap=353))
Chromosome(genes=[0, 5, 13, 21, 28, 34, 35, 39, 50, 52, 55, 57, 58, 60, 63, 64, 74, 75, 79, 91], fitness=Fitness(sortCount=20, gap=0))
4699
